Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

We want to maximize the fitness returned but minimize the number of calls.

First we will choose the genomes for reproduction (elitism, only pick the best)

Now we combine the genomes to create offsprings with crossovers (will have to tune the crossover rate)

We can also try to randomly flip some bits in the genomes (will have to tune mutation rate)

Evaluate the new offsprings only if it makes sense (i.e. if a genome was already evaluated we do not evaluate it)

And finally replace the worst performing with the new offsprings. Remember we want to minimze number of calls

In [23]:
from random import choices
from random import random
from random import randint
from random import sample
import lab9_lib

In [24]:
fitness = None
def create_problem(x):
    global fitness
    if fitness is not None:
        fitness = None  # dont know if this is needed but whatever
    fitness = lab9_lib.make_problem(x)  # create the problem instance


In [25]:
# 1st method to call to create the population
def create_population(size=10, genome_length=1000):
    '''List (of size=size) of 0 and 1 each of lenght = genome_lenght'''
    return [choices([0, 1], k=genome_length) for _ in range(size)]

In [26]:
already_evaluated_genomes = {}  # needed to check if the genome was already evaluated

# used to evaluate the current population
def evaluate_population(population: list):
    '''Returns a dictionary with the individual fitness of each genome'''
    # might modify this to return a list instead
    pop_fitness = {}
    global calls_saved
    for individual in population:
        individual = tuple(individual)  # convert to a tuple to be used as a key
        if already_evaluated_genomes.get(individual) is not None:   # if the individual was already tested
            pop_fitness[individual] = already_evaluated_genomes.get(individual)
            # prints the genome in string form and the fitness of said genome
            # print(f"{''.join(str(g) for g in individual)}: {already_evaluated_genomes.get(individual):.2%}")
            calls_saved += 1
        else:
            # fitness(ind) calls the __call__ function and passes genome
            ind_fit = fitness(individual)
            pop_fitness[individual] = ind_fit
            already_evaluated_genomes[individual] = ind_fit
            # prints the genome in string form and the fitness of said genome
            # print(f"{''.join(str(g) for g in individual)}: {ind_fit:.2%}")
    return pop_fitness  # returns a dictionary

In [27]:
# select the best genomes to mutate/crossover
def select_best_genomes(population: dict, size=4, pick_worse_prob = 0.25) -> dict:
    '''Receives as input a dictionary population, of which we only want to keep the most promising
    With a certain probability it can also pick random genomes instead.
    '''
    if len(population) >= 5:
        if random() < pick_worse_prob:  
            # literally just pick 4 random things
            # print("For this run, random genomes will be picked!")
            random_keys = sample(list(population.keys()), size)
            return {key : population[key] for key in random_keys}
        else: 
            # sort the dictionary based on the fitness, then pick the 4 most promising genomes
            # sorted returns a  list of tuples, so this one has to be kept under control because i am not sure lol
            # TODO CHECK HERE
            return dict(sorted(population.items(), key=lambda x: x[1], reverse=True)[:size])
    else:  
        # not worth selecting the best genomes yet
        return population

In [28]:
# method to call to print the population or to print the population + fitness after their evaluation
# ideally you will always call this after evaluatin the population btw
def print_population(population: list):
    if isinstance(population, dict):
        # handle dictionary logic
        for individual, fit in population.items():
            print(f"{''.join(str(g) for g in individual)}: {fit:.2%}")
    elif isinstance(population, list):
        for individual in population:
            print(f"{''.join(str(g) for g in individual)}")
    else:
        raise TypeError("Population must be a list or a dictionary")

In [29]:
def mutate(population: dict, mutation_rate=0.1) -> dict: 
    '''
    Each bit of each genome has a mutation_rate probability of switching
    returns a new dictionary after mutating the bits, see evaluate_population
    '''
    mutated_population = [] # population yet to be evaluated
    for individual, _ in population.items():
        # switch the bits if it's less than mutation_rate
        new_genome = (bit if random() > mutation_rate else 1 - bit for bit in individual)
        mutated_population.append(new_genome)
    
    return evaluate_population(mutated_population)

In [30]:
def crossover(population: dict) -> dict:
    '''
    Create new offsprings by splitting two parents from a random bit.
    Notice that this method doubles the population so a new call to select_best_population might be good
    '''
    new_population = []
    old_population = list(population.keys())
    for i in range(0, len(old_population), 2):
        genitore1 = old_population[i]
        # next one in the population if possible otherwise the first element will be genitore2
        genitore2 = old_population[i+1] if i+1 < len(old_population) else population[0]
        # randint includes the extremes but in this case we do not want the, so we go from 1 to -1
        crossover_point = randint(1, len(genitore1)-1)
        # the end point is not included, starting point is included
        offspring1 = genitore1[:crossover_point] + genitore2[crossover_point:]
        offspring2 = genitore2[:crossover_point] + genitore1[crossover_point:]
        new_population.append(offspring1)
        new_population.append(offspring2)
    return evaluate_population(new_population)

In [31]:
def run_genetic_algo(x, population_size=50, genome_length=1000, mutation_rate=0.05, desired_fitness = 50.00, max_stagnant_generations=100, convergence_threshold=0.01):
    best_genome = None
    best_fitness = float('-inf')
    last_best_fitness = float('-inf')
    generations = 0
    stagnant_generations = 0
    
    create_problem(x)    # create the problem instance
    population = create_population(population_size, genome_length)  # create the population
    #print_population(population)
    
    while best_fitness <= desired_fitness and stagnant_generations < max_stagnant_generations:  # until we reach a certain fitness
        fitnesses = evaluate_population(population)   # evaluate the current genomes
        #print_population(fitnesses)
        best_genomes = select_best_genomes(fitnesses)  # select the best genomes (or randomly pick them)
        #print_population(best_genomes)
        
        # they say you want to crossover first and then mutate
        crossed_genomes = crossover(best_genomes)
        mutated_genomes = mutate(crossed_genomes, mutation_rate)
        # print_population(mutated_genomes)
        
        current_best_genome, current_best_fitness = max(mutated_genomes.items(), key = lambda x : x[1])
        # print(best_genome, best_fitness)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_genome = current_best_genome
            stagnant_generations = 0
        else:   # implement a check to see how many generations the fitness did not improve at all
            stagnant_generations += 1
            
        # implement a check to see if the fitness is barely improving
        fitness_improvement = best_fitness - last_best_fitness
        if fitness_improvement < convergence_threshold:
            stagnant_generations += 1
            
        generations += 1
        
        if fitness.calls % 200000 == 0: 
            print(fitness.calls, best_fitness)
            
    return best_genome, best_fitness, fitness.calls, generations

In [32]:
mutation_rate = 0.01  # prob of each bit switching
calls_saved = 0  # just used to check if we save any calls
population_size = 50
desired_fitness = 0.9
x = [1, 2, 5, 10]
genome_length = 1000
max_stagnant_generations = 500
convergence_threshold = 0.01

In [33]:
best_genome, best_fitness, num_calls, generations = run_genetic_algo(
    x=5,
    population_size=population_size,
    genome_length=genome_length,
    mutation_rate=mutation_rate,
    desired_fitness=desired_fitness,
    max_stagnant_generations=max_stagnant_generations,
    convergence_threshold=convergence_threshold,
)
print(best_genome, best_fitness)
print(num_calls, generations)

(0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 